# Constrain Keff for maximum growth rate

In [1]:
import pandas
import numpy
from Bio import SeqIO
import re
import coralme
pandas.set_option('display.max_colwidth', None)
%load_ext autoreload
%autoreload 2

## Naive model

In [2]:
model = coralme.io.pickle.load_pickle_me_model("./pputidame/MEModel-step3-pputida-TS.pkl")

### M9

In [3]:
medium = pandas.read_csv("./media/M9.txt",index_col="ID",comment="#")["Bound"]
def enforce_medium(model,medium):
    for r in model.reactions.query("^EX_"):
        r.lower_bound = 0
    for r,lb in medium.items():
        model.reactions.get_by_id(r).lower_bound = lb
enforce_medium(model,medium)
assert model.feasibility() == True

In [4]:
model.reactions.get_by_id("EX_glc__D_e").lower_bound = -1000

In [5]:
model.optimize(max_mu=1.0)

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.5000000000000000	Optimal
        2	0.7500000000000000	Not feasible
        3	0.6250000000000000	Optimal
        4	0.6875000000000000	Not feasible
        5	0.6562500000000000	Not feasible
        6	0.6406250000000000	Not feasible
        7	0.6328125000000000	Optimal
        8	0.6367187500000000	Not feasible
        9	0.6347656250000000	Optimal
       10	0.6357421875000000	Not feasible
       11	0.6352539062500000	Not feasible
       12	0.6350097656250000	Not feasible
       13	0.6348876953125000	Optimal
       14	0.6349487304687500	Not feasible
       15	0.6349182128906250	Not feasible
       16	0.6349029541015625	Not feasible
       17	0.6348953247070312	Not feasible
       18	0.6348915100097656	Not feasible
       19	0.6348896026611328	Not feasible
       20	0.6348886489868164	Optimal


True

In [6]:
model

Name,pputida
Memory address,7a989c070310
Number of metabolites,7528
Number of reactions,14418
Number of genes,1677
Number of mRNA genes,1580
Number of rRNA genes,22
Number of tRNA genes,75
Objective expression,dummy_reaction_FWD_SPONT
Compartments,"ME-model Constraint, Cytoplasm, Extracellular, Periplasm, im, m, om"


In [7]:
from coralme.util.flux_analysis import exchange_single_model,flux_based_reactions
exchange_single_model(model).sort_values("flux",ascending=False).head(20)

,name,reaction,lower_bound,upper_bound,flux
id,,,,,
EX_h2o_e,,1.0 h2o_e <=>,-100,999999.0,4.195111e+01
EX_co2_e,,1.0 co2_e -->,0,999999.0,2.471629e+01
EX_h_e,,1.0 h_e <=>,-100,999999.0,6.349591e+00
biomass_dilution,,1.0 biomass -->,mu,mu,6.348886e-01
EX_ac_e,,1.0 ac_e -->,0,999999.0,3.904307e-01
DM_5drib_c,,1.0 5drib_c -->,0.0,999999.0,4.309287e-04
DM_tripeptide_c,,1.0 tripeptide_c -->,0.0,999999.0,1.858929e-04
DM_amob_c,,1.0 amob_c -->,0.0,999999.0,1.477683e-04
DM_doxopa_c,,1.0 doxopa_c -->,0.0,999999.0,1.415802e-04


## Keff sweep

In [8]:
def set_keff(model,keff):
    for r in model.reactions:
        if not isinstance(r,coralme.core.reaction.MetabolicReaction):
            continue
        r.keff = keff
        r.update()

In [ ]:
GrowthRates = {}
for keff in range(15,40,5):
    print(keff)
    set_keff(model,keff)
    model.optimize(max_mu=1.0)
    GrowthRates[keff] = model.solution.fluxes["biomass_dilution"]
GrowthRates = pandas.Series(GrowthRates)

15
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.5000000000000000	Not feasible
        2	0.2500000000000000	Optimal
        3	0.3750000000000000	Not feasible
        4	0.3125000000000000	Not feasible
        5	0.2812500000000000	Optimal
        6	0.2968750000000000	Optimal
        7	0.3046875000000000	Optimal
        8	0.3085937500000000	Optimal
        9	0.3105468750000000	Optimal
       10	0.3115234375000000	Not feasible
       11	0.3110351562500000	Not feasible
       12	0.3107910156250000	Not feasible
       13	0.3106689453125000	Not feasible
       14	0.3106079101562500	Not feasible
       15	0.3105773925781250	Not feasible
       16	0.3105621337890625	Not feasible
       17	0.3105545043945312	Optimal
       18	0.3105583190917969	Not feasible
       19	0.3105564117431641	Optimal
       20	0.3105573654174805	Optimal
20
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	0.500000000000000

In [ ]:
GrowthRates.to_csv("./datasets/growth-rates-per-keff.csv")

In [ ]:
m = (GrowthRates[35]-GrowthRates[30])/(35-30) 
b = GrowthRates[30] - m*30
Keff = (0.576-b)/m

In [ ]:
Keff

In [ ]:
set_keff(model,31)
coralme.io.pickle.save_pickle_me_model(model,"./pputidame/MEModel-step3-pputida-Keff.pkl")